In [98]:
import boto3
from datetime import datetime

In [99]:
training_job_name = 'sagemaker-pytorch-2021-01-17-02-36-21-851'

In [100]:
client = boto3.client("sagemaker")

In [101]:
role = 'arn:aws:iam::977053370764:role/service-role/AmazonSageMaker-ExecutionRole-20201202T141643'

In [102]:
training_job = client.describe_training_job(TrainingJobName=training_job_name)

In [103]:
training_job

{'TrainingJobName': 'sagemaker-pytorch-2021-01-17-02-36-21-851',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:977053370764:training-job/sagemaker-pytorch-2021-01-17-02-36-21-851',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-17-02-36-21-851/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'batch-size': '50',
  'dropout': '0.25',
  'epochs': '25',
  'hidden-layer': '50',
  'input-layer': '7',
  'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"sagemaker-pytorch-2021-01-17-02-36-21-851"',
  'sagemaker_program': '"train.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-17-02-36-21-851/source/sourcedir.tar.gz"',
  'seed': '1903'},
 'AlgorithmSpecification': {'TrainingImage': '520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-pytorch:0.4.0-gpu-py3',
  'Train

In [104]:
training_job["HyperParameters"]["sagemaker_submit_directory"].replace("\"", "")

's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-17-02-36-21-851/source/sourcedir.tar.gz'

In [105]:
training_job["ModelArtifacts"]["S3ModelArtifacts"]

's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-17-02-36-21-851/output/model.tar.gz'

In [106]:
model_creation = client.create_model(
    ModelName=training_job_name,
    PrimaryContainer={
        'ContainerHostname': training_job_name,
        'Image': training_job["AlgorithmSpecification"]["TrainingImage"],
        'ImageConfig': {
            'RepositoryAccessMode': 'Platform'
        },
        'Mode': 'SingleModel',
        'ModelDataUrl': training_job["ModelArtifacts"]["S3ModelArtifacts"],
        'Environment': {
              "SAGEMAKER_PROGRAM": "train.py",
              'SAGEMAKER_SUBMIT_DIRECTORY': training_job["HyperParameters"]["sagemaker_submit_directory"].replace("\"", "")
          },
    },
    ExecutionRoleArn=role)

In [107]:
config_creation = client.create_endpoint_config(
    EndpointConfigName=training_job_name,
    ProductionVariants=[
        {
            'VariantName': training_job_name,
            'ModelName': training_job_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium'
        },
    ]
)

In [108]:
endpoint_creation = client.create_endpoint(
    EndpointName=training_job_name,
    EndpointConfigName=training_job_name
)

In [109]:
import os
import io
import pandas as pd

In [110]:
ibovespa = pd.read_csv(os.path.join("data", "data.csv"))

In [111]:
with open(os.path.join("data", "data.csv")) as f:
    dados = f.read().encode("utf-8")

In [112]:
runtime = boto3.client("sagemaker-runtime")

In [114]:
response = runtime.invoke_endpoint(EndpointName='sagemaker-pytorch-2021-01-17-02-36-21-851',
                       Body=dados,
                       ContentType='text/csv',
                       Accept='Accept')

In [117]:
response["Body"].read().decode("utf-8")

'[0.10484547168016434, 0.2182859480381012, 0.3236715495586395, 0.4115184545516968, 0.4766795337200165, 0.5254829525947571, 0.5601202249526978, 0.5821685791015625, 0.5982837080955505, 0.6081803441047668, 0.6124605536460876, 0.6142750978469849, 0.6132918000221252, 0.607500433921814, 0.6003987193107605, 0.5944706797599792, 0.5902760624885559, 0.5876886248588562, 0.5857018232345581, 0.58204185962677, 0.5782886147499084, 0.5774833559989929, 0.5808299779891968, 0.5858021378517151, 0.5915355682373047, 0.5967764854431152, 0.6023174524307251, 0.6091407537460327, 0.6158046722412109, 0.6216511726379395, 0.6281389594078064, 0.6340134143829346, 0.6387987732887268, 0.6433814764022827, 0.6474251747131348, 0.6501380205154419, 0.6528744101524353, 0.6562739610671997, 0.6589218378067017, 0.6605839729309082, 0.6614805459976196, 0.6599161028862, 0.6578516960144043, 0.6577481627464294, 0.6554138660430908, 0.6506258249282837, 0.6461991667747498, 0.6417202353477478, 0.6382236480712891, 0.6362022161483765, 0.6